In [1]:
import numpy as np
import math

In [2]:
#reading data points 1
pts = np.zeros((1,2))
i=0
with open(r'./SpectData.txt') as file:
    for readline in file:
        line_strip = readline.strip() # removing \n
        point = line_strip.split('\t') # removing \t

        if(i==0):
            pts[0]=point
        else:
            pts=np.append(pts,np.array([point]),axis=0)  
        i+=1             
data=pts.astype(float)

data[0]
len(data)

2084

In [3]:



#reading data points 2
# pts2 = np.zeros((1,2))
# i=0
# with open(r'C:\Users\itisk\Desktop\File Storage\Workspace\1\ML\Data\SpectData2.txt') as file:
#     for readline in file:
#         line_strip = readline.strip() # removing \n
#         point = line_strip.split('\t') # removing \t

#         if(i==0):
#             pts2[0]=point
#         else:
#             pts2=np.append(pts2,np.array([point]),axis=0)  
#         i+=1             
# data2=pts.astype(float)

# data2[0]
# len(data2)

In [4]:
def weightFunc(pts,sigma):
    res=np.array([])
    for i in range(len(pts)):
        for j in range(i+1,len(pts)):
            calc=-1*(((pts[i][0]-pts[j][0])**2) + ((pts[i][1]-pts[j][1])**2))/2*((sigma*10**-2)**2)
            res=np.append(res,np.array([calc]),axis=0)
    print(sigma)
    res=np.exp(res)
    return(res)


In [5]:
y = weightFunc(data,1)

KeyboardInterrupt: 

In [ ]:
print(len(y))
print("===============================================")
print(y)

4950
